In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.impute import SimpleImputer,MissingIndicator, KNNImputer
from sklearn.model_selection import train_test_split, cross_validate,GridSearchCV,RepeatedStratifiedKFold,cross_val_score,cross_val_predict
from sklearn.preprocessing import PowerTransformer,OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.compose import make_column_transformer,make_column_selector,ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, GradientBoostingRegressor,RandomForestClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import roc_auc_score
from sklearn.impute import  KNNImputer
from sklearn.svm import SVC
from lightgbm import LGBMRegressor, LGBMClassifier
import xgboost as xgb

import warnings; warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
#from kuma_utils.preprocessing.imputer import LGBMImputer 
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler,RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
import holidays

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import SplineTransformer
from sklearn.preprocessing import PolynomialFeatures
from geopy.distance import geodesic 

import gc


from category_encoders import OrdinalEncoder


import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.preprocessing import KBinsDiscretizer,LabelEncoder
from feature_engine.encoding import OrdinalEncoder
from feature_engine.discretisation import EqualFrequencyDiscretiser
from feature_engine.encoding import RareLabelEncoder
from feature_engine.selection import DropDuplicateFeatures, DropConstantFeatures
from feature_engine.selection import (RecursiveFeatureElimination, RecursiveFeatureAddition,
    DropConstantFeatures,
    DropDuplicateFeatures,
    SmartCorrelatedSelection,DropCorrelatedFeatures
)
from feature_engine.creation import MathFeatures,RelativeFeatures

# to select the features
from sklearn.feature_selection import SelectKBest, SelectPercentile
from feature_engine.encoding import CountFrequencyEncoder
from feature_engine.encoding import OneHotEncoder
from feature_engine.encoding import MeanEncoder
from sklearn import metrics
from feature_engine.encoding import CountFrequencyEncoder
from sklearn.cluster import KMeans
from category_encoders import MEstimateEncoder,OrdinalEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import KBinsDiscretizer


from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor ,AdaBoostRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, BayesianRidge, ElasticNet, LassoLars, PassiveAggressiveRegressor
from sklearn.svm import SVR, NuSVR
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from catboost import CatBoostRegressor, metrics, Pool, cv
pd.set_option('display.max_columns', None)

In [2]:
import geopy.distance

In [11]:

train = pd.read_csv(r"C:\Users\91786\Desktop\DS\food_ds\data\extracted\ext.csv")
#val = pd.read_csv(r"C:\Users\91786\Desktop\DS\data\val.csv")

#print(f"Train shape: {train.shape}")
#print(f"Validation shape: {val.shape}")

In [14]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class DeliveryVisualization:
    """Class for creating advanced visualizations of delivery data"""
    
    def __init__(self, df):
        """Initialize with a dataframe that already has the necessary features"""
        self.df = df.copy()
        
        # Clean the target variable if needed
        if 'Time_taken(min)' in self.df.columns:
            if self.df['Time_taken(min)'].dtype == 'object':
                self.df['Time_taken(min)'] = self.df['Time_taken(min)'].str.extract(r'(\d+)').astype(float)
    
    def create_location_density_clusters(self, n_clusters=5):
        """Create location density clusters based on delivery coordinates"""
        print("Creating location density clusters...")
        
        # Create delivery location clusters
        coords = self.df[['Delivery_location_latitude', 'Delivery_location_longitude']].values
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        self.df['Location_Cluster'] = kmeans.fit_predict(coords)
        
        # Calculate density within each cluster
        nn = NearestNeighbors(n_neighbors=min(50, len(self.df)))
        nn.fit(coords)
        distances, _ = nn.kneighbors(coords)
        density = 1 / (distances.sum(axis=1) + 1e-6)  # Add small epsilon to avoid division by zero
        
        # Normalize density to 0-10 range for better visualization
        min_density, max_density = density.min(), density.max()
        normalized_density = 1 + 9 * (density - min_density) / (max_density - min_density)
        self.df['Location_Density'] = normalized_density
        
        # Get cluster centers for reference
        self.cluster_centers = kmeans.cluster_centers_
        
        print(f"Created {n_clusters} location clusters with density calculation")
        return self.df
    
    def create_3d_animated_visualization(self):
        """Create an animated 3D visualization of multiple deliveries, density and time taken"""
        print("Creating 3D animated visualization...")
        
        # Ensure we have location clusters and density
        if 'Location_Cluster' not in self.df.columns or 'Location_Density' not in self.df.columns:
            self.create_location_density_clusters()
        
        # Ensure multiple_deliveries is numeric
        self.df['multiple_deliveries'] = pd.to_numeric(self.df['multiple_deliveries'], errors='coerce').fillna(0)
        
        # If Time_Period doesn't exist, create a simple version
        if 'Time_Period' not in self.df.columns and 'Order_Hour' in self.df.columns:
            time_bins = [0, 6, 10, 12, 14, 17, 20, 23, 24]
            time_labels = ['Late_Night', 'Morning', 'Lunch', 'Early_Afternoon', 
                          'Late_Afternoon', 'Dinner', 'Evening', 'Night']
            self.df['Time_Period'] = pd.cut(
                self.df['Order_Hour'], 
                bins=time_bins, 
                labels=time_labels, 
                include_lowest=True
            )
        elif 'Time_Period' not in self.df.columns:
            # Fallback if Order_Hour doesn't exist
            self.df['Time_Period'] = 'All Day'
        
        # Prepare data - group by relevant dimensions
        grouped = self.df.groupby(['Location_Cluster', 'multiple_deliveries', 'Time_Period']).agg({
            'Time_taken(min)': 'mean',
            'Location_Density': 'mean',
            'Distance_km': 'mean' if 'Distance_km' in self.df.columns else lambda x: 2  # Default value if missing
        }).reset_index()
        
        # Create color map for clusters
        colors = px.colors.qualitative.Bold
        grouped['Cluster_Color'] = grouped['Location_Cluster'].apply(lambda x: colors[int(x) % len(colors)])
        
        # Create animation frames for each time period
        time_periods = grouped['Time_Period'].unique()
        
        # Create 3D plot with animation
        fig = go.Figure()
        
        # Add traces for each time period (for animation)
        for period in time_periods:
            period_data = grouped[grouped['Time_Period'] == period]
            
            # Handle bubble sizing based on what data is available
            marker_size = period_data['Distance_km'] * 2 if 'Distance_km' in period_data.columns else 5
            
            fig.add_trace(
                go.Scatter3d(
                    x=period_data['multiple_deliveries'],
                    y=period_data['Location_Density'],
                    z=period_data['Time_taken(min)'],
                    mode='markers',
                    marker=dict(
                        size=marker_size,
                        color=period_data['Cluster_Color'],
                        opacity=0.8
                    ),
                    text=[
                        f"Cluster: {c}<br>Time Period: {p}<br>Multiple Deliveries: {m}<br>"
                        f"Density: {d:.2f}<br>Avg Time: {t:.2f} min"
                        for c, p, m, d, t in zip(
                            period_data['Location_Cluster'],
                            period_data['Time_Period'],
                            period_data['multiple_deliveries'],
                            period_data['Location_Density'],
                            period_data['Time_taken(min)']
                        )
                    ],
                    hoverinfo="text",
                    name=period,
                    visible=False  # Start with all traces hidden for animation
                )
            )
        
        # Set first trace to be visible
        if len(fig.data) > 0:
            fig.data[0].visible = True
        
        # Create animation steps
        steps = []
        for i, period in enumerate(time_periods):
            step = dict(
                method="update",
                args=[{"visible": [False] * len(fig.data)},
                      {"title": f"Delivery Analysis: {period}"}],
                label=period
            )
            step["args"][0]["visible"][i] = True
            steps.append(step)
        
        sliders = [dict(
            active=0,
            currentvalue={"prefix": "Time Period: "},
            pad={"t": 50},
            steps=steps
        )]
        
        # Update layout
        fig.update_layout(
            title="3D Analysis of Multiple Deliveries, Location Density and Delivery Time",
            scene=dict(
                xaxis_title="Number of Multiple Deliveries",
                yaxis_title="Location Density (higher = denser)",
                zaxis_title="Avg. Time Taken (min)",
                xaxis=dict(range=[0, grouped['multiple_deliveries'].max() + 0.5]),
                yaxis=dict(range=[0, 10]),
                zaxis=dict(range=[0, grouped['Time_taken(min)'].max() * 1.1])
            ),
            height=800,
            width=1000,
            sliders=sliders
        )
        
        # Add a trend surface to show patterns
        x_range = np.linspace(0, grouped['multiple_deliveries'].max(), 10)
        y_range = np.linspace(1, 10, 10)
        X, Y = np.meshgrid(x_range, y_range)
        Z = np.zeros_like(X)
        
        # Simple model: more deliveries and higher density = more time
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                Z[i, j] = 15 + 2 * X[j, i] + 0.5 * Y[j, i]
        
        # Add the surface
        fig.add_trace(go.Surface(
            x=X, y=Y, z=Z,
            colorscale='Blues',
            opacity=0.3,
            showscale=False,
            name="Trend Surface"
        ))
        
        return fig

# Example usage in the notebook:
# viz = DeliveryVisualization(df=train_fe)  # Use your processed dataframe
# viz.create_location_density_clusters()  # Create clusters if needed
# fig = viz.create_3d_animated_visualization()
# fig.show()  # Display in notebook

In [15]:
# Create visualization object
viz = DeliveryVisualization(df=train)

# Create clusters
viz.create_location_density_clusters(n_clusters=5)

# Generate the figure
fig = viz.create_3d_animated_visualization()

# Display it
fig.show()

# Optional: Save to HTML
fig.write_html("delivery_visualization.html")

Creating location density clusters...
Created 5 location clusters with density calculation
Creating 3D animated visualization...


In [ ]:
train['Tim']

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_Order_picked_formatted,Time_Ordered_formatted,order_prepare_time_diff_mins,Distance_km,Restaurant_Cluster,Delivery_Cluster,Distance_From_City_Center,Delivery_Angle,Delivery_Cluster_Density,day,month,year,week,day_of_week,Order_Quarter,Is_Weekend,dayofyear,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,is_leap_year,days_in_month,Time_Orderd_dt,Time_Order_picked_dt,Pickup_Time_Mins,Order_Hour,Pickup_Hour,Time_Period,Is_Lunch_Rush,Is_Dinner_Rush,Is_Rush_Hour
0,0x4607,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30,11:45,Sunny,High,2,Snack,motorcycle,0.0,No,INDORES,2022-03-19 11:45:00,2022-03-19 11:30:00,15.0,3.020737,0,5,790.363668,0.807879,5213,19,3,2022,11,5,1,1,78,0,0,0,0,0,0,0,31,11:30:00,11:45:00,15.0,11,11,Morning,1,0,1
1,0xb379,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45,19:50,Stormy,Jam,2,Snack,scooter,1.0,No,BANGRES,2022-03-25 19:50:00,2022-03-25 19:45:00,5.0,20.143737,3,2,894.008674,-0.565529,13223,25,3,2022,12,4,1,0,84,0,0,0,0,0,0,0,31,19:45:00,19:50:00,5.0,19,19,Evening,0,1,1
2,0x5d6d,BANGRES19DEL01,23.0,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30,08:45,Sandstorms,Low,0,Drinks,motorcycle,1.0,No,BANGRES,2022-03-19 08:45:00,2022-03-19 08:30:00,15.0,1.549693,3,2,890.331660,-0.585887,13223,19,3,2022,11,5,1,1,78,0,0,0,0,0,0,0,31,08:30:00,08:45:00,15.0,8,8,Morning,0,0,0
3,0x7a6a,COIMBRES13DEL02,38.0,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00,18:10,Sunny,Medium,0,Buffet,motorcycle,1.0,No,COIMBRES,2022-04-05 18:10:00,2022-04-05 18:00:00,10.0,7.774497,3,2,973.403438,-0.803719,13223,5,4,2022,14,1,2,0,95,0,0,0,0,0,0,0,30,18:00:00,18:10:00,10.0,18,18,Afternoon,0,1,1
4,0x70a2,CHENRES12DEL01,32.0,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30,13:45,Cloudy,High,1,Snack,scooter,1.0,No,CHENRES,2022-03-26 13:45:00,2022-03-26 13:30:00,15.0,6.197898,3,2,1127.536324,-0.440538,13223,26,3,2022,12,5,1,1,85,0,0,0,0,0,0,0,31,13:30:00,13:45:00,15.0,13,13,Afternoon,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56987,0x6909,JAPRES01DEL01,35.0,4.6,26.905190,75.810753,27.015190,75.920753,2022-03-27,21:35,21:45,Sunny,Jam,1,Snack,scooter,1.0,No,JAPRES,2022-03-27 21:45:00,2022-03-27 21:35:00,10.0,16.365894,2,2,1157.485238,1.099075,1496,27,3,2022,12,6,1,1,86,0,0,0,0,0,0,0,31,21:35:00,21:45:00,10.0,21,21,Evening,0,1,1
56988,0x443b,JAPRES11DEL01,33.0,4.9,26.902940,75.793007,26.912940,75.803007,2022-03-11,11:40,11:45,Sandstorms,High,1,Drinks,scooter,1.0,No,JAPRES,2022-03-11 11:45:00,2022-03-11 11:40:00,5.0,1.488108,2,2,1142.159290,1.104660,1496,11,3,2022,10,4,1,0,70,0,0,0,0,0,0,0,31,11:40:00,11:45:00,5.0,11,11,Morning,1,0,1
56989,0x1ea5,SURRES11DEL03,NaN,NaN,21.157735,72.768778,21.217735,72.828778,2022-03-11,00:00,21:05,NaN,NaN,3,Drinks,scooter,1.0,No,SURRES,2022-03-11 21:05:00,NaN,NaN,9.107659,4,3,442.687177,1.128763,3074,11,3,2022,10,4,1,0,70,0,0,0,0,0,0,0,31,00:00:00,21:05:00,1265.0,0,21,NaN,0,0,0
56990,0x22d4,VADRES03DEL02,27.0,4.7,22.320000,73.170000,22.450000,73.300000,2022-03-06,18:35,18:40,Sandstorms,Medium,0,Meal,motorcycle,0.0,No,VADRES,2022-03-06 18:40:00,2022-03-06 18:35:00,5.0,19.658584,4,3,587.167066,1.147672,3074,6,3,2022,9,6,1,1,65,0,0,0,0,0,0,0,31,18:35:00,18:40:00,5.0,18,18,Afternoon,0,1,1


In [22]:
train

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,0x7c09,JAPRES04DEL01,30,4.8,26.902328,75.794257,26.912328,75.804257,24-03-2022,11:35:00,11:45:00,conditions Windy,High,1,Meal,motorcycle,0,No,Metropolitian,(min) 32
45589,0xd641,AGRRES16DEL01,21,4.6,0.000000,0.000000,0.070000,0.070000,16-02-2022,19:55:00,20:10:00,conditions Windy,Jam,0,Buffet,motorcycle,1,No,Metropolitian,(min) 36
45590,0x4f8d,CHENRES08DEL03,30,4.9,13.022394,80.242439,13.052394,80.272439,11-03-2022,23:50:00,00:05:00,conditions Cloudy,Low,1,Drinks,scooter,0,No,Metropolitian,(min) 16
45591,0x5eee,COIMBRES11DEL01,20,4.7,11.001753,76.986241,11.041753,77.026241,07-03-2022,13:35:00,13:40:00,conditions Cloudy,High,0,Snack,motorcycle,1,No,Metropolitian,(min) 26


In [23]:
val = val.replace('NaN', float(np.nan), regex=True)
train=train.replace('NaN', float(np.nan), regex=True)

### converting to correct data types

In [ ]:


def clean_and_transform_data(df: pd.DataFrame, val: bool) -> pd.DataFrame:
    """
    Cleans and transforms the dataset:
    - Extracts actual weather condition
    - Removes 'min' from Time_taken(min) and converts to float
    - Converts date and time columns correctly
    - Computes order preparation time in minutes
    """


    df.columns = df.columns.str.strip()
    if 'Weatherconditions' in df.columns:
        df['Weatherconditions'] = df['Weatherconditions'].str.split(" ", expand=True).iloc[:, -1]
    if not val and 'Time_taken(min)' in df.columns:
        df['Time_taken(min)'] = df['Time_taken(min)'].str.extract('(\d+)').astype(float)
    num_cols = ['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude', 'Restaurant_longitude',
                'Delivery_location_latitude', 'Delivery_location_longitude', 'Vehicle_condition', 'multiple_deliveries', 'Time_taken(min)']
    
    for col in num_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    if 'Order_Date' in df.columns:
        df['Order_Date'] = pd.to_datetime(df['Order_Date'], format="%d-%m-%Y", errors='coerce')
    if 'Time_Orderd' in df.columns:
        df['Time_Orderd'] = pd.to_timedelta(df['Time_Orderd'], errors='coerce')
    
    if 'Time_Order_picked' in df.columns:
        df['Time_Order_picked'] = pd.to_timedelta(df['Time_Order_picked'], errors='coerce')
    if {'Time_Orderd', 'Time_Order_picked', 'Order_Date'}.issubset(df.columns):
        df['Time_Order_picked_formatted'] = np.where(df['Time_Order_picked'] < df['Time_Orderd'],
                                                     df['Order_Date'] + pd.DateOffset(1) + df['Time_Order_picked'],
                                                     df['Order_Date'] + df['Time_Order_picked'])
        
        df['Time_Ordered_formatted'] = df['Order_Date'] + df['Time_Orderd']
        df['order_prepare_time_diff_mins'] = (df['Time_Order_picked_formatted'] - df['Time_Ordered_formatted']).dt.total_seconds() / 60

    return df

train = clean_and_transform_data(train, val=False)
val = clean_and_transform_data(val, val=True)



In [25]:
train

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min),Time_Order_picked_formatted,Time_Ordered_formatted,order_prepare_time_diff_mins
0,0x4607,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,0 days 11:30:00,...,2,Snack,motorcycle,0.0,No,Urban,24.0,2022-03-19 11:45:00,2022-03-19 11:30:00,15.0
1,0xb379,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,0 days 19:45:00,...,2,Snack,scooter,1.0,No,Metropolitian,33.0,2022-03-25 19:50:00,2022-03-25 19:45:00,5.0
2,0x5d6d,BANGRES19DEL01,23.0,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,0 days 08:30:00,...,0,Drinks,motorcycle,1.0,No,Urban,26.0,2022-03-19 08:45:00,2022-03-19 08:30:00,15.0
3,0x7a6a,COIMBRES13DEL02,38.0,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,0 days 18:00:00,...,0,Buffet,motorcycle,1.0,No,Metropolitian,21.0,2022-04-05 18:10:00,2022-04-05 18:00:00,10.0
4,0x70a2,CHENRES12DEL01,32.0,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,0 days 13:30:00,...,1,Snack,scooter,1.0,No,Metropolitian,30.0,2022-03-26 13:45:00,2022-03-26 13:30:00,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,0x7c09,JAPRES04DEL01,30.0,4.8,26.902328,75.794257,26.912328,75.804257,2022-03-24,0 days 11:35:00,...,1,Meal,motorcycle,0.0,No,Metropolitian,32.0,2022-03-24 11:45:00,2022-03-24 11:35:00,10.0
45589,0xd641,AGRRES16DEL01,21.0,4.6,0.000000,0.000000,0.070000,0.070000,2022-02-16,0 days 19:55:00,...,0,Buffet,motorcycle,1.0,No,Metropolitian,36.0,2022-02-16 20:10:00,2022-02-16 19:55:00,15.0
45590,0x4f8d,CHENRES08DEL03,30.0,4.9,13.022394,80.242439,13.052394,80.272439,2022-03-11,0 days 23:50:00,...,1,Drinks,scooter,0.0,No,Metropolitian,16.0,2022-03-12 00:05:00,2022-03-11 23:50:00,15.0
45591,0x5eee,COIMBRES11DEL01,20.0,4.7,11.001753,76.986241,11.041753,77.026241,2022-03-07,0 days 13:35:00,...,0,Snack,motorcycle,1.0,No,Metropolitian,26.0,2022-03-07 13:40:00,2022-03-07 13:35:00,5.0


In [29]:
def geospatial_features(df):
    """Extract geospatial features"""
    print("Extracting geospatial features...")
    
    df = df.copy()
    df['City'] = df['Delivery_person_ID'].str.extract(r'([A-Z]+)')


    required_cols = [
        'Restaurant_latitude', 'Restaurant_longitude', 
        'Delivery_location_latitude', 'Delivery_location_longitude'
    ]
    
    
    def haversine_distance(row):
        try:
            restaurant = (row['Restaurant_latitude'], row['Restaurant_longitude'])
            delivery = (row['Delivery_location_latitude'], row['Delivery_location_longitude'])
            return geopy.distance.geodesic(restaurant, delivery).km
        except:
            return np.nan
    
    df['Distance_km'] = df.apply(haversine_distance, axis=1)
  
    restaurant_coords = df[['Restaurant_latitude', 'Restaurant_longitude']].values
    kmeans_restaurant = KMeans(n_clusters=5, random_state=42)
    df['Restaurant_Cluster'] = kmeans_restaurant.fit_predict(restaurant_coords)
    
    # Delivery location clusters
    delivery_coords = df[['Delivery_location_latitude', 'Delivery_location_longitude']].values
    kmeans_delivery = KMeans(n_clusters=8, random_state=42)
    df['Delivery_Cluster'] = kmeans_delivery.fit_predict(delivery_coords)
    
    # Distance from city center (assuming the mean location is roughly the city center)
    city_center_lat = df['Delivery_location_latitude'].mean()
    city_center_lon = df['Delivery_location_longitude'].mean()
    
    def distance_from_center(row):
        try:
            delivery = (row['Delivery_location_latitude'], row['Delivery_location_longitude'])
            center = (city_center_lat, city_center_lon)
            return geopy.distance.geodesic(delivery, center).km
        except:
            return np.nan
    
    df['Distance_From_City_Center'] = df.apply(distance_from_center, axis=1)
    
    # Create polar coordinates for cyclical patterns
    df['Delivery_Angle'] = np.arctan2(
        df['Delivery_location_latitude'] - city_center_lat,
        df['Delivery_location_longitude'] - city_center_lon
    )

    delivery_cluster_counts = df['Delivery_Cluster'].value_counts().to_dict()
    df['Delivery_Cluster_Density'] = df['Delivery_Cluster'].map(delivery_cluster_counts)
    
    return df

In [30]:
train_fe = geospatial_features(train)
val_fe = geospatial_features(val)

Extracting geospatial features...
Extracting geospatial features...


In [31]:
train_fe

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Time_taken(min),Time_Order_picked_formatted,Time_Ordered_formatted,order_prepare_time_diff_mins,Distance_km,Restaurant_Cluster,Delivery_Cluster,Distance_From_City_Center,Delivery_Angle,Delivery_Cluster_Density
0,0x4607,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,0 days 11:30:00,...,24.0,2022-03-19 11:45:00,2022-03-19 11:30:00,15.0,3.020737,0,5,790.363668,0.807879,5213
1,0xb379,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,0 days 19:45:00,...,33.0,2022-03-25 19:50:00,2022-03-25 19:45:00,5.0,20.143737,3,2,894.008674,-0.565529,13223
2,0x5d6d,BANGRES19DEL01,23.0,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,0 days 08:30:00,...,26.0,2022-03-19 08:45:00,2022-03-19 08:30:00,15.0,1.549693,3,2,890.331660,-0.585887,13223
3,0x7a6a,COIMBRES13DEL02,38.0,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,0 days 18:00:00,...,21.0,2022-04-05 18:10:00,2022-04-05 18:00:00,10.0,7.774497,3,2,973.403438,-0.803719,13223
4,0x70a2,CHENRES12DEL01,32.0,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,0 days 13:30:00,...,30.0,2022-03-26 13:45:00,2022-03-26 13:30:00,15.0,6.197898,3,2,1127.536324,-0.440538,13223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,0x7c09,JAPRES04DEL01,30.0,4.8,26.902328,75.794257,26.912328,75.804257,2022-03-24,0 days 11:35:00,...,32.0,2022-03-24 11:45:00,2022-03-24 11:35:00,10.0,1.488112,4,6,1164.022390,1.087448,5172
45589,0xd641,AGRRES16DEL01,21.0,4.6,0.000000,0.000000,0.070000,0.070000,2022-02-16,0 days 19:55:00,...,36.0,2022-02-16 20:10:00,2022-02-16 19:55:00,15.0,10.983242,1,1,7976.251994,-2.900591,3640
45590,0x4f8d,CHENRES08DEL03,30.0,4.9,13.022394,80.242439,13.052394,80.272439,2022-03-11,0 days 23:50:00,...,16.0,2022-03-12 00:05:00,2022-03-11 23:50:00,15.0,4.648024,3,2,1123.851354,-0.437815,13223
45591,0x5eee,COIMBRES11DEL01,20.0,4.7,11.001753,76.986241,11.041753,77.026241,2022-03-07,0 days 13:35:00,...,26.0,2022-03-07 13:40:00,2022-03-07 13:35:00,5.0,6.219668,3,2,974.356746,-0.804667,13223


In [ ]:

def time_features(df):
    """Extract time-based features from Order_Date and Time_Orderd/Time_Order_picked"""
    print("Extracting time features...")

    df = df.copy()

    df['Order_Date'] = pd.to_datetime(df['Order_Date'], format='%d-%m-%Y')

    # Extract date components
    df['day'] = df['Order_Date'].dt.day
    df['month'] = df['Order_Date'].dt.month
    df['year'] = df['Order_Date'].dt.year
    df['week'] = df['Order_Date'].dt.isocalendar().week
    df['day_of_week'] = df['Order_Date'].dt.dayofweek
    df['Order_Quarter'] = df['Order_Date'].dt.quarter
    df['Is_Weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

    # Additional date features
    df["dayofyear"] = df.Order_Date.dt.dayofyear
    df["is_month_start"] = df.Order_Date.dt.is_month_start.astype(int)
    df["is_month_end"] = df.Order_Date.dt.is_month_end.astype(int)
    df["is_quarter_start"] = df.Order_Date.dt.is_quarter_start.astype(int)
    df["is_quarter_end"] = df.Order_Date.dt.is_quarter_end.astype(int)
    df["is_year_start"] = df.Order_Date.dt.is_year_start.astype(int)
    df["is_year_end"] = df.Order_Date.dt.is_year_end.astype(int)
    df["is_leap_year"] = df.Order_Date.dt.is_leap_year.astype(int)
    df["days_in_month"] = df.Order_Date.dt.days_in_month

    for col in ['Time_Orderd', 'Time_Order_picked']:
        if col in df.columns:
            df[col] = df[col].astype(str).fillna('00:00')

          
            df[col] = df[col].apply(lambda x: str(x).split()[-1][:5] if "days" in str(x) else str(x))

            df[col] = df[col].apply(
                lambda x: datetime.strptime(x, '%H:%M').strftime('%H:%M') if ':' in x else '00:00'
            )

    if 'Time_Orderd' in df.columns and 'Time_Order_picked' in df.columns:
        df['Time_Orderd_dt'] = pd.to_datetime(df['Time_Orderd'], format='%H:%M').dt.time
        df['Time_Order_picked_dt'] = pd.to_datetime(df['Time_Order_picked'], format='%H:%M').dt.time

        def time_diff_minutes(row):
            if pd.isnull(row['Time_Orderd_dt']) or pd.isnull(row['Time_Order_picked_dt']):
                return np.nan

            t1 = datetime.combine(datetime.today(), row['Time_Orderd_dt'])
            t2 = datetime.combine(datetime.today(), row['Time_Order_picked_dt'])

            if t2 < t1:
                t2 += timedelta(days=1)

            return (t2 - t1).total_seconds() / 60

        df['Pickup_Time_Mins'] = df.apply(time_diff_minutes, axis=1)

        df['Order_Hour'] = pd.to_datetime(df['Time_Orderd'], format='%H:%M').dt.hour
        df['Pickup_Hour'] = pd.to_datetime(df['Time_Order_picked'], format='%H:%M').dt.hour

        df['Time_Period'] = pd.cut(
            df['Order_Hour'],
            bins=[0, 6, 12, 18, 24],
            labels=['Night', 'Morning', 'Afternoon', 'Evening']
        )

        df['Is_Lunch_Rush'] = ((df['Order_Hour'] >= 11) & (df['Order_Hour'] <= 13)).astype(int)
        df['Is_Dinner_Rush'] = ((df['Order_Hour'] >= 18) & (df['Order_Hour'] <= 21)).astype(int)
        df['Is_Rush_Hour'] = ((df['Is_Lunch_Rush'] == 1) | (df['Is_Dinner_Rush'] == 1)).astype(int)

    return df

train_fe = time_features(train_fe)
val_fe = time_features(val_fe)


Extracting time features...
Extracting time features...


In [36]:
train

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min),Time_Order_picked_formatted,Time_Ordered_formatted,order_prepare_time_diff_mins
0,0x4607,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,0 days 11:30:00,...,2,Snack,motorcycle,0.0,No,Urban,24.0,2022-03-19 11:45:00,2022-03-19 11:30:00,15.0
1,0xb379,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,0 days 19:45:00,...,2,Snack,scooter,1.0,No,Metropolitian,33.0,2022-03-25 19:50:00,2022-03-25 19:45:00,5.0
2,0x5d6d,BANGRES19DEL01,23.0,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,0 days 08:30:00,...,0,Drinks,motorcycle,1.0,No,Urban,26.0,2022-03-19 08:45:00,2022-03-19 08:30:00,15.0
3,0x7a6a,COIMBRES13DEL02,38.0,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,0 days 18:00:00,...,0,Buffet,motorcycle,1.0,No,Metropolitian,21.0,2022-04-05 18:10:00,2022-04-05 18:00:00,10.0
4,0x70a2,CHENRES12DEL01,32.0,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,0 days 13:30:00,...,1,Snack,scooter,1.0,No,Metropolitian,30.0,2022-03-26 13:45:00,2022-03-26 13:30:00,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,0x7c09,JAPRES04DEL01,30.0,4.8,26.902328,75.794257,26.912328,75.804257,2022-03-24,0 days 11:35:00,...,1,Meal,motorcycle,0.0,No,Metropolitian,32.0,2022-03-24 11:45:00,2022-03-24 11:35:00,10.0
45589,0xd641,AGRRES16DEL01,21.0,4.6,0.000000,0.000000,0.070000,0.070000,2022-02-16,0 days 19:55:00,...,0,Buffet,motorcycle,1.0,No,Metropolitian,36.0,2022-02-16 20:10:00,2022-02-16 19:55:00,15.0
45590,0x4f8d,CHENRES08DEL03,30.0,4.9,13.022394,80.242439,13.052394,80.272439,2022-03-11,0 days 23:50:00,...,1,Drinks,scooter,0.0,No,Metropolitian,16.0,2022-03-12 00:05:00,2022-03-11 23:50:00,15.0
45591,0x5eee,COIMBRES11DEL01,20.0,4.7,11.001753,76.986241,11.041753,77.026241,2022-03-07,0 days 13:35:00,...,0,Snack,motorcycle,1.0,No,Metropolitian,26.0,2022-03-07 13:40:00,2022-03-07 13:35:00,5.0


In [37]:
data = pd.concat([train_fe.drop("Time_taken(min)",axis=1), val_fe], axis=0)
y = train_fe['Time_taken(min)']
data.to_csv(r"C:\Users\91786\Desktop\DS\data\data.csv", index=False)
y.to_csv(r"C:\Users\91786\Desktop\DS\data\y.csv", index=False)